In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [2]:
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more'
)

word2idx = {w: i for i, w in enumerate(list(set(sentence.split())))}
idx2word = {i: w for i, w in enumerate(list(set(sentence.split())))}
n_class = len(word2idx) # classification problem
max_len = len(sentence.split())
n_hidden = 5


In [3]:
def make_data(sentence):
    input_batch = []
    target_batch = []

    words = sentence.split() # ['Github', 'Actions', 'makes', ...]
    for i in range(max_len - 1): # i = 2
        input = [word2idx[n] for n in words[:(i + 1)]] # input = [18 7 3]
        input = input + [0] * (max_len - len(input)) # input = [18 7 3 0 'it', ..., 0]
        target = word2idx[words[i + 1]] # target = [0]
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

# input_batch: [max_len - 1, max_len, n_class]
input_batch, target_batch = make_data(sentence)
print(input_batch.shape)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 16, True)

torch.Size([20, 21, 19])


In [4]:
class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden,batch_first=True, bidirectional=True)
        # fc
        self.fc = nn.Linear(n_hidden * 2, n_class)

    def forward(self, X):
        # X: [batch_size, max_len, n_class]
        batch_size = X.shape[0]
        hidden_state = torch.randn(1*2, batch_size, n_hidden).cuda()   # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        cell_state = torch.randn(1*2, batch_size, n_hidden).cuda()     # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        outputs, (_, _) = self.lstm(X, (hidden_state, cell_state)) # [batch_size, max_len, n_hidden * 2]
        outputs = outputs[:,-1,:]  # [batch_size, n_hidden * 2]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model

model = BiLSTM().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [5]:
# Training
for epoch in range(10000):
    for x, y in loader:
        pred = model(x.cuda())
        loss = criterion(pred, y.cuda())
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 1000 cost = 1.827255
Epoch: 1000 cost = 2.262238
Epoch: 2000 cost = 1.392085
Epoch: 2000 cost = 1.468587
Epoch: 3000 cost = 0.989689
Epoch: 3000 cost = 1.386539
Epoch: 4000 cost = 0.703100
Epoch: 4000 cost = 0.588614
Epoch: 5000 cost = 0.457731
Epoch: 5000 cost = 0.449804
Epoch: 6000 cost = 0.286784
Epoch: 6000 cost = 0.528751
Epoch: 7000 cost = 0.282234
Epoch: 7000 cost = 0.224325
Epoch: 8000 cost = 0.254053
Epoch: 8000 cost = 0.158323
Epoch: 9000 cost = 0.213200
Epoch: 9000 cost = 0.190479
Epoch: 10000 cost = 0.175502
Epoch: 10000 cost = 0.204196


In [10]:
model.eval()
# Pred
predict = model(input_batch.cuda()).data.max(1, keepdim=True)[1]
print(sentence)
print([idx2word[n.item()] for n in predict.squeeze()])

GitHub Actions makes it easy to automate all your software workflows from continuous integration and delivery to issue triage and more
['Actions', 'makes', 'it', 'easy', 'to', 'automate', 'all', 'your', 'software', 'workflows', 'from', 'continuous', 'integration', 'and', 'and', 'to', 'issue', 'triage', 'and', 'and']
